In [67]:
%%capture
%pip install numpy as np
%pip install pandas as pd
%pip install sklearn


In [68]:
import numpy as np 
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
df = pd.read_csv('dataset/GHRP_dataset_csv/baseline.csv')

In [ ]:
raw_features = df.iloc[:, 1:-1].values.astype(float)
y = df.iloc[:,-1].values
test_size = 0.2
x_train, x_test,y_train,y_test = train_test_split(raw_features,y,test_size=test_size,random_state=42,stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)



0.4995871180842279


In [ ]:

def sigmoid(z):
    return np.where(z >= 0, 
                    1 / (1 + np.exp(-z)), 
                    np.exp(z) / (1 + np.exp(z)))


def train_model(X,y,learning_rate,theta,theta_zero,epochs,reg):
    n, d = X.shape
    y = y.reshape(-1,1)
    

    for i in range(epochs):
        z = np.dot(X,theta) + theta_zero
        p = sigmoid(z)

        error = p-y


        d_theta = (1/n)*np.dot(X.T, error) + reg*np.sign(theta)
        d_theta_zero = (1/n)*np.sum(p-y)

        theta -= learning_rate*d_theta
        theta_zero -= learning_rate*d_theta_zero
    return theta, theta_zero

## Training

In [102]:
def calculate_accuracy(X_test,y_test,theta,theta_zero):
    z = np.dot(X_test,theta) + theta_zero
    y_test=y_test.reshape(-1,1)
    predictions = (sigmoid(z)>=0.5)
    accuracy = (predictions==y_test).mean()



    print(f"Percentage of predicted 1s: {predictions.mean():.2%}")
    return accuracy


In [103]:
n, d = X_train.shape

eta = 0.2
theta = np.zeros(d).reshape(-1,1)
theta_zero = 0 
l1_reg = 0.001
epochs = 20000

theta,theta_zero = train_model(X_train,y_train,eta,theta,theta_zero,epochs,l1_reg)

/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/ipykernel_97434/1188043561.py:3: RuntimeWarning: overflow encountered in exp
  1 / (1 + np.exp(-z)),
/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/ipykernel_97434/1188043561.py:4: RuntimeWarning: overflow encountered in exp
  np.exp(z) / (1 + np.exp(z)))
/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/ipykernel_97434/1188043561.py:4: RuntimeWarning: invalid value encountered in divide
  np.exp(z) / (1 + np.exp(z)))
/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/ipykernel_97434/1188043561.py:19: RuntimeWarning: overflow encountered in square
  d_theta = (1/n)*np.dot(X.T, error) + reg*(theta**2)


In [ ]:
accuracy = calculate_accuracy(X_test,y_test,theta,theta_zero)
print("Accuracy:", round(accuracy*100,2),"%") 

Percentage of predicted 1s: 79.44%
Accuracy: 64.08


/var/folders/br/qs2w2dkd2zxf5nl4znds0jkm0000gn/T/ipykernel_97434/1188043561.py:4: RuntimeWarning: invalid value encountered in divide
  np.exp(z) / (1 + np.exp(z)))


In [105]:
from IPython.display import display, Markdown

md_table = '### Feature Importance for Defect Prediction \n'
md_table += 'Metric Name | Weight Magnitude | Impact \n'
md_table += ' | :--- | :--- | :--- \n'

feature_names = df.columns[:-1]
print(feature_names)
importance = sorted(list(zip(feature_names,abs(theta),np.sign(theta))), key=lambda x: abs(x[1]), reverse=True)

for name, weight, sign in importance :

    impact = '✖ Little Impact' if (weight < 1e-3) else ('⬆ Probability' if sign>0 else '⬇ Probability')
    md_table += f"| {name} | {weight:.4f} | {impact}\n"

display(Markdown(md_table))

Index(['SHA', 'cbo', 'wmc', 'dit', 'rfc', 'lcom', 'totalMethods',
       'totalFields', 'nosi', 'loc', 'returnQty', 'loopQty', 'comparisonsQty',
       'tryCatchQty', 'parenthesizedExpsQty', 'stringLiteralsQty',
       'numbersQty', 'assignmentsQty', 'mathOperationsQty', 'variablesQty',
       'maxNestedBlocks', 'uniqueWordsQty'],
      dtype='object')


TypeError: unsupported format string passed to numpy.ndarray.__format__